In [31]:
import pandas as pd
import torch
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import logging
import joblib

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [33]:
def evaluate_model(model, X_test, y_test, device):
    
    try:
        model.eval()  # Set the model to evaluation mode
        with torch.no_grad():  # Disable gradient calculation
            # Convert test data to PyTorch tensors and move to the appropriate device
            X_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(device)
            # Add sequence dimension
            X_tensor = X_tensor.unsqueeze(1)

            # Make predictions
            outputs = model(X_tensor)
            _, predicted = torch.max(outputs, 1)

            # Move predictions and true labels to CPU for metric calculation
            y_pred = predicted.cpu().numpy()
            y_true = y_test.values

        # Calculate evaluation metrics
        accuracy = accuracy_score(y_true, y_pred)
        classification_rep = classification_report(y_true, y_pred)
        confusion_mat = confusion_matrix(y_true, y_pred)

        # Store the metrics in a dictionary
        metrics = {
            'accuracy': accuracy,
            'classification_report': classification_rep,
            'confusion_matrix': confusion_mat
        }

        logging.info("Model evaluation completed successfully.")
        return metrics

    except Exception as e:
        logging.error(f"Error during model evaluation: {e}")
        return None

In [34]:
import main

def load_test_data(file_path, preprocessor_path="models/preprocessed_data.joblib", scaler_path="models/scaler.joblib"):
    try:
        # Load the saved preprocessor
        preprocessor = joblib.load(preprocessor_path)  # Load the preprocessor object
        if not isinstance(preprocessor, main.NIDSPreprocessor):
            raise ValueError("Loaded object is not an instance of NIDSPreprocessor.")

        # Load the data using the preprocessor's load_data method
        data = preprocessor.load_data(file_path)

        # Extract features and labels
        X_test = data.drop('label', axis=1)
        y_test = data['label']

        # Transform the test data using the preprocessor's transform method
        X_test_transformed = preprocessor.transform(X_test)

        logging.info("Test data loaded and preprocessed successfully.")
        return X_test_transformed, y_test

    except FileNotFoundError as e:
        logging.error(f"File not found: {e}")
        return None, None
    except Exception as e:
        logging.error(f"Error loading or preprocessing test data: {e}")
        return None, None

In [35]:
def load_trained_model(model_path, input_dim, device, model_class):
    
    try:
        # Initialize the model with the appropriate input dimension
        model = model_class(input_dim=input_dim)
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.to(device)  # Move the model to the specified device
        model.eval()  # Set the model to evaluation mode
        logging.info("Trained model loaded successfully.")
        return model
    except FileNotFoundError as e:
        logging.error(f"Model file not found: {e}")
        return None
    except Exception as e:
        logging.error(f"Error loading model: {e}")
        return None


In [36]:
def print_metrics(metrics):
    
    if metrics:
        print("Evaluation Metrics:")
        print(f"Accuracy: {metrics['accuracy']:.4f}")
        print("\nClassification Report:\n", metrics['classification_report'])
        print("\nConfusion Matrix:\n", metrics['confusion_matrix'])
    else:
        print("No metrics to print.")

In [37]:
if __name__ == "__main__":
    # Configurations
    TEST_DATA_FILE = 'data/UNSW-NB15_4.csv'  # Path to your test dataset
    MODEL_PATH = "models/transformer_model.pth"  # Path to your trained model
    PREPROCESSOR_PATH = "models/preprocessed_data.joblib"  # Path to the preprocessor
    SCALER_PATH = "models/scaler.joblib"
    INPUT_DIM = 43  # Input dimension of your model
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available
    MODEL_CLASS = None
    try:
        import main
        MODEL_CLASS = main.TransformerNIDS
    except Exception as e:
        print("error loading main model: " + str(e))

    # Load test data
    X_test, y_test = load_test_data(TEST_DATA_FILE, PREPROCESSOR_PATH, SCALER_PATH)
    if X_test is None or y_test is None:
        print("Failed to load test data.")
    elif MODEL_CLASS is None:
        print("Model class not loaded.")
    else:
        # Load the trained model
        trained_model = load_trained_model(MODEL_PATH, INPUT_DIM, DEVICE, MODEL_CLASS)

        if trained_model:
            # Evaluate the model
            metrics = evaluate_model(trained_model, X_test, y_test, DEVICE)

            # Print the evaluation metrics
            print_metrics(metrics)
        else:
            print("Failed to load the trained model.")

2025-03-31 07:43:56,604 - ERROR - Error loading or preprocessing test data: Loaded object is not an instance of NIDSPreprocessor.


Failed to load test data.
